Some Python modules that may come in handy:

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
from scipy.spatial.distance import cosine

For explanatory purposes, we will create a frequency matrix here (rather than an inverted index).

In [2]:
x = np.array([[1,0,2,0,0], [0,1,1,0,0], [0,1,1,0,1],[0,0,0,1,1]])
index = pd.DataFrame(x, index=["auto", "car", "wash", "machine"], columns=[0,1,2,3,4])
index

,0,1,2,3,4
auto,1,0,2,0,0
car,0,1,1,0,0
wash,0,1,1,0,1
machine,0,0,0,1,1


In [3]:
index[1]

auto       0
car        1
wash       1
machine    0
Name: 1, dtype: int64

In [4]:
index.loc['auto']

0    1
1    0
2    2
3    0
4    0
Name: auto, dtype: int64

In [5]:
index.shape

(4, 5)

## Inverse document frequency

We can now implement the inverse document frequency. The overall number of documents is the number of columns in the matrix. The *df* of a term is the number of non-zeros in the term's row.

In [6]:
def idf(index, term):
    _, n_docs = index.shape
    df = np.count_nonzero(index.loc[term])
    return math.log(float(n_docs) / df)

In [7]:
print(idf(index, "car"), idf(index, "auto"), idf(index, "wash"), idf(index, "machine"))

0.9162907318741551 0.9162907318741551 0.5108256237659907 0.9162907318741551


## Dot product

Since we need to be able to compute the dot-product of two vectors for both the numerator and the Euclidean norms in the denominator. Let's try this!

In [8]:
def simple_dot(a, b):
    dsum = 0.
    for ((idx,), val) in np.ndenumerate(a):
        dsum += float(val) * float(b[idx])
    return dsum

Let's check if it works on two document vectors:

In [9]:
simple_dot(index[1], index[2])

2.0

We do not really need this implementation, since numpy provides a dot product function as well:

In [10]:
np.dot(index[1], index[2])

2

## Cosine similarity

We almost have all the bits and pieces to compute the cosine similarity between a document and a vector. Let's write two helper functions. One computes TF-IDF, the other converts a query to a vector:

In [11]:
def l2_norm(a):
    return math.sqrt(np.dot(a, a))

def cosine_similarity(a, b):
    return np.dot(a,b) / (l2_norm(a) * l2_norm(b))

cosine_similarity(np.array([1,2,3,4]), np.array([1,2,3,4]))

1.0

Computing the Euclidean norm and cosine similarity is such a common operation that most scientific Python packages implement it for you.

## Query processing

We almost have all the bits and pieces to process a query. First, we make functions to compute the TF-IDF and to construct a query vector:

In [13]:
def tfidf(tf, idf):
    return tf * idf

def query_vector(index, query_terms):
    query_terms = set(query_terms)
    n_terms, _ = index.shape
    query_vector = np.zeros(n_terms)
    
    for idx, term in enumerate(index.index):
        if term in query_terms:
            query_vector[idx] = idf(index, term)

    return query_vector
        
query_vector(index, ["car", "wash"])

array([ 0.        ,  0.91629073,  0.51082562,  0.        ])

Now we proceed with the query processing:

    * First we compute the query vector.
    * We compute per document:
        * The vector of TF-IDFs.
        * The cosine similarity between this vector and the query vector.
    

In [14]:
def query(index, query_terms):
    q = query_vector(index, query_terms)
    n_terms, _ = index.shape

    results = []
    for doc in index:
        doc_vec = np.zeros(n_terms)
    
        for (idx, (term, tf)) in enumerate(index[doc].iteritems()):
            doc_vec[idx] = tfidf(tf, idf(index, term))   
        results.append((doc, cosine_similarity(q, doc_vec)))

    return sorted(results, key=lambda t: t[1], reverse=True)

In [15]:
query(index, ["car", "wash"])

[(1, 1.0),
 (2, 0.49680738410267594),
 (4, 0.23710617314601054),
 (0, 0.0),
 (3, 0.0)]

In [16]:
query(index, ["car", "auto"])

[(2, 0.92050541877203973),
 (0, 0.70710678118654757),
 (1, 0.61761388700950914),
 (3, 0.0),
 (4, 0.0)]

In [17]:
query(index, ["car"])

[(1, 0.8734379353188122),
 (2, 0.43393041582178132),
 (0, 0.0),
 (3, 0.0),
 (4, 0.0)]

## Inverted index

Remembed our index:

In [18]:
index

,0,1,2,3,4
auto,1,0,2,0,0
car,0,1,1,0,0
wash,0,1,1,0,1
machine,0,0,0,1,1


* What are the differences between this index and the inverted index?
* How can we compute the Euclidean norms?